In [1]:
# === Standard Imports ===
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
from IPython.display import display

# === Project Path Setup ===
import sys
from pathlib import Path
sys.path.append(str(Path.home() / "work" / "EMS_QI_Projects" / "ahaems-2025-submission"))

# === Project Resources ===
from project_paths import BASE_DIR, FALLOUTS_DIR, REPORTS_DIR, print_project_paths
from config import load_cleaned_data

# === Load Cleaned Data ===
print("📥 Loading data from PostgreSQL (ahaems_cleaned)...")
df = load_cleaned_data()

# === Rename for internal use ===
df = df.rename(columns={
    "UniqueIncidentKey": "incident_id",
    "Patient Age (ePatient.15)": "age",
    "Patient Age Units (ePatient.16)": "age_units",
    "Primary Impression": "primary_impression",
    "Secondary Impression": "secondary_impression",
    "Transport Disposition": "transport_disposition",
    "Response Type Of Service Requested With Code (eResponse.05)": "response_type",
    "Situation Last Known Well Date Time (eSituation.18)": "lkw_time",
    "Vitals Signs Taken Date Time (eVitals.01)": "vitals_time",
    "Patient Cincinnati Stroke Scale Used (eVitals.30)": "stroke_scale_type",
    "Patient Initial Stroke Scale Score (eVitals.29)": "stroke_scale_score",
    "Cardiac Arrest During EMS Event With Code (eArrest.01)": "cardiac_arrest",
    "Disposition Final Patient Acuity Code (eDisposition.19)": "final_acuity"
})

# === Convert datetime fields ===
df["lkw_time"] = pd.to_datetime(df["lkw_time"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")
df["vitals_time"] = pd.to_datetime(df["vitals_time"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")

# === Extract ICDs ===
def extract_icd_prefix(text):
    if isinstance(text, str):
        match = re.search(r"\(([A-Z]\d{2}(?:\.\d+)?)\)", text.upper())
        if match:
            return match.group(1)[:3]
    return ""

df["primary_icd"] = df["primary_impression"].apply(extract_icd_prefix)
df["secondary_icd"] = df["secondary_impression"].apply(extract_icd_prefix)

# === Aggregate to 1 row per incident ===
grouped = df.groupby("incident_id").agg({
    "age": "first",
    "age_units": "first",
    "primary_icd": "first",
    "secondary_icd": "first",
    "transport_disposition": "first",
    "response_type": "first",
    "vitals_time": "min",
    "lkw_time": "first",
    "stroke_scale_type": "first",
    "stroke_scale_score": "first",
    "cardiac_arrest": "first",
    "final_acuity": "first"
}).reset_index()

# === Quarter Extraction ===
grouped["quarter"] = grouped["vitals_time"].dt.to_period("Q")

# === Denominator Logic ===
grouped["age"] = pd.to_numeric(grouped["age"], errors="coerce")
age_valid = (grouped["age"] >= 18) & (grouped["age_units"].str.lower() == "years")
stroke_icds = ["I60", "I61", "I63", "G45", "G46"]
impression_valid = grouped["primary_icd"].isin(stroke_icds) | grouped["secondary_icd"].isin(stroke_icds)
transport_valid = grouped["transport_disposition"].str.contains("transport by", case=False, na=False)
response_valid = grouped["response_type"].str.contains("2205001|2205003|2205009", na=False)

# Exclusions
exclude_lkw = (
    grouped["lkw_time"].notna() & grouped["vitals_time"].notna() &
    ((grouped["vitals_time"] - grouped["lkw_time"]).dt.total_seconds() >= 86400)
)
exclude_arrest = grouped["cardiac_arrest"].astype(str).isin(["3001003", "3001005"])
exclude_acuity = grouped["final_acuity"].astype(str) == "4219909"

grouped["in_denominator"] = (
    age_valid & impression_valid & transport_valid & response_valid &
    ~exclude_lkw & ~exclude_arrest & ~exclude_acuity
)

# === Valid values for numerator ===
valid_stroke_scores = [
    "3329001 - Negative",
    "3329003 - Non-Conclusive",
    "3329005 - Positive"
]

# === Numerator Logic (simplified for local practice) ===
grouped["in_numerator"] = (
    grouped["in_denominator"] &
    (grouped["stroke_scale_type"].astype(str).str.lower() == "true") &
    grouped["stroke_scale_score"].isin(valid_stroke_scores)
)

# === Debug output: Unique values after normalization ===
print("✅ Stroke Scale Score Value Counts:")
print(grouped["stroke_scale_score"].value_counts(dropna=False))

print("\n🧠 Inclusion Check Summary:")
print("In Denominator:", grouped["in_denominator"].sum())
print("In Numerator:", grouped["in_numerator"].sum())

# === Summary Output ===
summary = (
    grouped[grouped["in_denominator"]]
    .groupby("quarter")
    .agg(
        AHAEMS4_Denominator=("in_denominator", "sum"),
        AHAEMS4_Numerator=("in_numerator", "sum")
    )
    .reset_index()
)
summary["AHAEMS4_Percentage"] = (summary["AHAEMS4_Numerator"] / summary["AHAEMS4_Denominator"] * 100).round(2)
display(summary)

# === Plot AHA Measure 1 Line Graph ===
plt.figure(figsize=(10, 5))
plt.plot(summary["quarter"].astype(str), summary["AHAEMS4_Percentage"], marker="o", linestyle="-", label="AHAEMS4 %")
plt.title("AHA EMS Measure 4 – 12 Lead ECG Acquisition ")
plt.xlabel("Quarter")
plt.ylabel("Percentage (%)")
plt.ylim(0, 100)
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

# === Export Fallout CSV ===
fallouts = grouped[grouped["in_denominator"] & ~grouped["in_numerator"]]
fallout_path = FALLOUTS_DIR / "ahaems44_fallouts.csv"

# Ensure directory exists before writing
os.makedirs(fallout_path.parent, exist_ok=True)

# Write fallout file
fallouts.to_csv(fallout_path, index=False)

from project_paths import OUTPUT_DIR

# Save the summary chart to output/charts/
chart_path = OUTPUT_DIR / "charts" / "ahaems4_chart.png"
os.makedirs(chart_path.parent, exist_ok=True)
plt.savefig(chart_path, bbox_inches="tight")

🔧 Detected Environment: e879fd4b227d
📂 BASE_DIR: /home/jovyan/work/EMS_QI_Projects/ahaems-2025-submission
📁 DATA_RAW_DIR: /home/jovyan/work/EMS_QI_Projects/ahaems-2025-submission/data/raw
📁 DATA_CLEANED_DIR: /home/jovyan/work/EMS_QI_Projects/ahaems-2025-submission/data/cleaned
📁 FALLOUTS_DIR: /home/jovyan/work/EMS_QI_Projects/ahaems-2025-submission/output/fallouts
📁 REPORTS_DIR: /home/jovyan/work/EMS_QI_Projects/ahaems-2025-submission/output/reports
📁 NOTEBOOKS_DIR: /home/jovyan/work/EMS_QI_Projects/ahaems-2025-submission/notebooks
📁 SCRIPTS_DIR: /home/jovyan/work/EMS_QI_Projects/ahaems-2025-submission/scripts
📥 Loading data from PostgreSQL (ahaems_cleaned)...
✅ Stroke Scale Score Value Counts:
stroke_scale_score
None                        28878
3329001 - Negative          15428
3329003 - Non-Conclusive      763
3329005 - Positive            525
Name: count, dtype: int64

🧠 Inclusion Check Summary:
In Denominator: 756
In Numerator: 658


,quarter,AHAEMS4_Denominator,AHAEMS4_Numerator,AHAEMS4_Percentage
0,2024Q1,151,141,93.38
1,2024Q2,149,136,91.28
2,2024Q3,64,57,89.06
3,2024Q4,155,148,95.48
4,2025Q1,139,108,77.70
5,2025Q2,97,68,70.10
